In [6]:
import os

import numpy as np
import pandas as pd

import scipy
from scipy.sparse import csc_matrix, csr_matrix, load_npz

import matplotlib.pyplot as plt

from chord_rec_lib import delta_t #find_files, format_name

In [7]:
from chord_rec_lib import dnames

HEAD_DIR = '..'
for d in dnames:
    dnames[d] = os.path.join(HEAD_DIR, dnames[d])

# Load data

In [19]:
X_ds = pd.read_csv(os.path.join(dnames['CSVS_DIR'], 'X_ds.csv'), index_col=0)
X_ds.head()

,song_id,fullname,starttime,endtime,chord_id,duration,id
0,906,E:(1),119.730158,119.780948,100,0.05079,0
1,906,E:(1),119.780948,119.831738,100,0.05079,1
2,906,E:(1),119.831738,119.882528,100,0.05079,2
3,906,E:(1),119.882528,119.933318,100,0.05079,3
4,906,E:(1),119.933318,119.984108,100,0.05079,4
